In [ ]:
import os
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
offline.init_notebook_mode(connected=True)

def create_graphs(data_frames, **kwargs):
    data = [        
            go.Scatter(
                x=data_frames[df_index][data_frames[df_index].columns[0]],
                y=data_frames[df_index][data_frames[df_index].columns[i]],
                opacity=1.0, 
                hoverinfo="none",
                xaxis=("x" + str(df_index + 1)),
                yaxis=("y" + str(df_index + 1)),
                name=data_frames[df_index].columns[i]
            ) for df_index in range(0, len(data_frames)) 
            for i in range(1,data_frames[df_index].columns.size)
    ]
    
    layout = go.Layout(**kwargs)
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False, "showLink":False})

def create_graph(data_frame, title=None, x_axis_title=None, y_axis_title = None):
    data = [        
            go.Scatter(
                x=data_frame[data_frame.columns[0]],
                y=data_frame[data_frame.columns[i]],
                opacity=1.0,
                name=data_frame.columns[i],
                hoverinfo="none",
            ) 
            for i in range(1,data_frame.columns.size)
            
    ]
    
    #fig = tools.make_subplots(rows=1, cols=len(data_frames))
    layout = go.Layout(
        title=title,         
        showlegend=False,
        xaxis=dict(
            title=x_axis_title
        ),
        yaxis=dict(
            title=y_axis_title
        )        
    )
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False, "showLink":False})
    
def create_surface_plot(matrix, title=None):
    data = [
        go.Surface(
            z=matrix
        )
    ]
    
    layout = go.Layout(
        title=title,         
        showlegend=False,
    )
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": True, "showLink":True})

In [ ]:
from simcore_sdk.nodeports.nodeports import PORTS

In [ ]:
# from ECG_1D.m

import pandas as pd


def plot_ECG():
    data_path_a = PORTS.inputs[0].get()    
    data_frame_a = pd.read_csv(data_path_a, sep="\t", header=None)
    # membrane potential
    axis_colums = [0,1]
    plot_data = data_frame_a.filter(items=[data_frame_a.columns[i] for i in axis_colums])
    create_graph(data_frame=plot_data)

In [ ]:
import numpy as np
from numpy import ndarray
import pandas as pd
from pandas import DataFrame

def ap_surface_1D():
    data_path_a = PORTS.inputs[0].get()
    data_frame_a = pd.read_csv(data_path_a, sep="\t", header=None)
    
    axis_colums = [0,1]
    plot_data = data_frame_a.filter(items=[data_frame_a.columns[i] for i in axis_colums])
    create_graph(data_frame=plot_data)
    
    data_path_str = PORTS.inputs[1].get()
    data_frame_JJ = pd.read_csv(data_path_str, sep="\t", header=None)
    out_v = data_frame_JJ
        
    num_cells = 165
    # point ranges you want to plot
    min = 0 #90000;
    max = 999 #100000;
    diff = (max - min + 1)

    
    t = np.array(out_v.iloc[range(min,max+1), 0]).reshape(diff, 1)
    T = t
    #creates a matrix of t matricies (1 for each cell)
    for b in range(1,(num_cells)):
        T = np.hstack((T, t))
    
    
    #creats an array of cell numbers
    cellnum = np.array([x for x in range(1, num_cells+1)]).reshape(1, num_cells)
    
    #creates a square matrix of cell numbers on the diagonal       
    cellm = np.ones((diff,1)) @ cellnum
    vm = out_v.iloc[range(min,max+1), range(1,num_cells+1)]
    
    colormap = [
        [0, 'rgb(0,0,0)'],
        [1.0, 'rgb(180.0, 180.0, 180.0)']
    ]
    
    data = [
        go.Surface(           
            x=cellm,
            y=T,
            z=vm.values,
            colorscale=colormap,
            reversescale=True
        )
    ]
    
    camera = dict(
        up=dict(x=-1, y=0, z=0),
        center=dict(x=0,y=0,z=0),
        eye=dict(x=1, y=0, z=2)
    )
    
    layout = go.Layout(
        title=None,
        scene=dict(camera=camera)
    )
    fig = go.Figure(data=data, layout=layout)
    
    
    offline.iplot(fig)

In [ ]:
ap_surface_1D()